# Collecting and filtering received headlines

## Collect Headline files

In [ ]:
import pandas as pd
import numpy as np
import glob
import datetime
from datetime import time


#path = r'C:\DRO\DCL_rawdata_files' # use your path
path = '' # use your path
all_files = glob.glob("4_WRDS_SP 500 Headlines completed/Single Files/*.csv")

li = []

# Load all files
for filename in all_files:
    df = pd.read_csv(filename)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=False)

In [ ]:
frame

In [ ]:
# convert versionCreated to datetime
frame.versionCreated =pd.to_datetime(frame.versionCreated)
frame

In [ ]:
# Set index to version created without timestamp
frame.set_index(frame['versionCreated'].dt.normalize(),inplace=True)

## Filter Headlines

In [ ]:
# Drop original 'index' timestamp
frame.drop(columns=['Unnamed: 0'],inplace=True)
frame

In [ ]:
sum(frame["RIC"]=="APA.O")

In [ ]:
# Change APA.0 to APA (fix that a different Ticker was used for same company)
frame["RIC"].replace({"APA.O": "APA"}, inplace=True)

In [ ]:
sum(frame["RIC"]=="APA.O")

In [ ]:
# Sort dataframe by index
frame = frame.sort_index()

In [ ]:
# Remove duplicates
frame= frame.drop_duplicates()
frame

In [ ]:
frame[frame.sourceCode== "NS:EDG"]

In [ ]:
frame[frame.sourceCode== "NS:GLFILE"]

In [ ]:
frame[frame.sourceCode== "NS:TRANS"]

In [ ]:
# Filter out specific news sources
frame=frame[frame.sourceCode!= "NS:EDG"]
frame=frame[frame.sourceCode!= "NS:TRANS"]
frame=frame[frame.sourceCode!= "NS:GLFILE"]
frame

In [ ]:
# remove headlines with wrong tagging of CBOE.K unitl end of June 2019
frame[((frame.sourceCode== "NS:PUBT") & (frame.RIC=="CBOE.K") & (frame.index<"2020-06-30"))]

In [ ]:
# remove headlines with wrong tagging of CBOE.K unitl end of June 2019
frame=frame[~((frame.sourceCode== "NS:PUBT") & (frame.RIC=="CBOE.K") & (frame.index<"2020-06-30"))]
frame

In [ ]:
frame[frame.RIC== "MCO"]

In [ ]:
# remove headlines for Moodys
frame=frame[frame.RIC!= "MCO"]
frame

In [ ]:
frame.index.max()

In [ ]:
# Filter out February / July news
frame=frame.loc['2019-03-01':'2020-06-30']
frame

In [ ]:
frame.tail(10)

In [ ]:
# Store frame into dataframe
frame.to_csv("4_WRDS_SP 500 Headlines completed/2019_03_01_to_2020_06_30_Headlines_SP500.csv")

In [ ]:
# create flag column that concatenates text, sourcecode and RIC
frame["Duplicate_Flag"]= (frame.text+frame.sourceCode+frame.RIC).str.lower()
frame

In [ ]:
# find day with duplicates headlines
frame.loc["2020-06-01"].sort_values("text",ascending=False)

In [ ]:
# rename index label
frame.index.rename('versionCreated.1',inplace=True)
frame

In [ ]:
# sort by version created
frame = frame.sort_values(by=frame.columns[0])
frame

In [ ]:
frame.drop_duplicates("Duplicate_Flag",inplace=True)
frame

In [ ]:
# check again day with duplicates headlines
frame.loc["2020-06-01"].sort_values("text",ascending=False)

In [ ]:
frame=frame.drop("Duplicate_Flag",axis=1)
frame

In [ ]:
# Check how many news are on weekends
import holidays

us_holidays = holidays.UnitedStates()

In [ ]:
from datetime import date
# As the expand parameter of uk_holidays= True, the list of holidays will expand
# for each year that is checked. Therfore, we generate a loop to effectivley initialise all 
# holidays from 2000- 2020 in the holiday object
for x in range (2019,2020):
  date(x, 1, 1) in us_holidays

# Check length of holiday object
len(us_holidays)

In [ ]:
frame["Holiday_Flag"]=frame.index.isin(us_holidays)
frame

In [ ]:
frame[frame.Holiday_Flag==True]

In [ ]:
# Filter out holidays
frame=frame[frame.Holiday_Flag==False]
frame

In [ ]:
dict_days={0:"Mon",
          1:"Tue",
          2:"Wed",
          3:"Thu",
          4:"Fri",
          5:"Sat",
          6:"Sun"}


# Insert controll variable for the day of the week (0=Monday, 6=Sunday)
frame["Weekday"]=pd.to_datetime(frame.index).dayofweek.map(dict_days)
frame

In [ ]:
frame[frame.Weekday.isin(["Sat","Sun"])]

In [ ]:
frame=frame[~frame.Weekday.isin(["Sat","Sun"])]
frame

In [ ]:
frame=frame.drop(columns=["Holiday_Flag","Weekday"])
frame

In [ ]:
print(662739-8772)

In [ ]:
# Store frame into dataframe
frame.to_csv("4_WRDS_SP 500 Headlines completed/2019_03_01_to_2020_06_30_Headlines_SP500_filt.csv")

### Analysis of news headlines

In [ ]:
frame["storyId"].value_counts() 

In [ ]:
# unique news
frame["storyId"].nunique()

In [ ]:
# Create dataframe containing unique storyIds
news_Ids=frame[["storyId","sourceCode"]]
print(news_Ids.shape)
news_Ids=news_Ids.drop_duplicates()
print(news_Ids.shape)

# Store dataframe containing only unique headlines
news_Ids.to_csv("4_WRDS_SP 500 Headlines completed/2019_03_01_to_2020_04_30_unique_HeadlineIds_SP100.csv")

In [ ]:
# Rough monthly estimate
frame["storyId"].nunique()/13

In [ ]:
# Print latest news date
max(frame.versionCreated)

In [ ]:
# average news frequency per company per day
frame["storyId"].nunique()/505/370

In [ ]:
frame["month_year"]=frame.versionCreated.dt.strftime('%Y-%m')
frame["month_year_day"]=frame.versionCreated.dt.strftime('%Y-%m-%d')



# Generate pivot table of sum of news per day per company
piv_table_news = pd.pivot_table(frame,index=['RIC'],columns=[frame.month_year],\
               values=["storyId"],\
               aggfunc=['count'],fill_value=0)

# Display the table
piv_table_news

In [ ]:
# Store in excel
piv_table_news.to_excel("4_WRDS_SP 500 Headlines completed/Overview News per Company and month.xlsx")

Analysis Daily averages

In [ ]:
frame

In [ ]:
# Analysis daily averages

# Generate pivot table of sum of news per day per company
piv_table_news_daily = pd.pivot_table(frame,index=['RIC'],columns=[frame.month_year_day],\
               values=["storyId"],\
               aggfunc=['count'],fill_value=0)

# Display the table
piv_table_news_daily

In [ ]:
piv_table_news_daily[piv_table_news_daily.index=="CBOE.K"].to_excel("check_CBOE.K_new_vol.xlsx")
piv_table_news[piv_table_news.index=="CBOE.K"].to_excel("check_CBOE.K_new_vol_monthly.xlsx")

In [ ]:
frame[frame.RIC=="CBOE.K"].reset_index()[["month_year_day","text","sourceCode","RIC"]].to_excel("CBOE.K News.xlsx")

In [ ]:
#check only news for PUBT 
frame[frame.sourceCode=="NS:PUBT"].reset_index()[["month_year_day","text","sourceCode","RIC"]].to_excel("PUBT News.xlsx")

In [ ]:
piv_table_news_daily_sum= pd.DataFrame()
piv_table_news_daily_sum["Mean"]=piv_table_news_daily.mean(axis=1)
piv_table_news_daily_sum["Median"]=piv_table_news_daily.median(axis=1)
piv_table_news_daily_sum["Max"]=piv_table_news_daily.max(axis=1)
piv_table_news_daily_sum["Count_days"]=piv_table_news_daily.count(axis=1)
piv_table_news_daily_sum

In [ ]:
# calculate days with no articles
piv_table_news_daily_sum["days_with_no_articles"]=(piv_table_news_daily == 0).sum(axis=1)
piv_table_news_daily_sum

In [ ]:
# calculate % of days with no articles
piv_table_news_daily_sum["days_with_no_articles_PERC"]=piv_table_news_daily_sum["days_with_no_articles"]/piv_table_news_daily_sum["Count_days"]
piv_table_news_daily_sum

In [ ]:
# Store in excel
piv_table_news_daily_sum.to_excel("4_WRDS_SP 500 Headlines completed/Overview News count per Company and day.xlsx")

Analysis by Source

In [ ]:
frame["month_year"]=frame.versionCreated.dt.strftime('%Y-%m')

# Generate pivot table of sum news per author
piv_table_authors = pd.pivot_table(frame,index=['sourceCode'],columns=[frame.month_year],\
               values=["storyId"],\
               aggfunc=['count'],fill_value=0)

# Display the table
piv_table_authors

In [ ]:
# Store in excel
piv_table_authors.to_excel("4_WRDS_SP 500 Headlines completed/Overview News per Author and month.xlsx")

In [ ]:
# Check how many news are on weekends
import holidays

us_holidays = holidays.UnitedStates()

In [ ]:
from datetime import date
# As the expand parameter of uk_holidays= True, the list of holidays will expand
# for each year that is checked. Therfore, we generate a loop to effectivley initialise all 
# holidays from 2000- 2020 in the holiday object
for x in range (2019,2020):
  date(x, 1, 1) in us_holidays

# Check length of holiday object
len(us_holidays)

In [ ]:
frame["Holiday_Flag"]=frame.index.isin(us_holidays)
frame

In [ ]:
frame.loc["2019-05-27"]

In [ ]:
frame[frame.Holiday_Flag==True]

In [ ]:
for ptr in holidays.UnitedStates(years = 2019).items(): 
    print(ptr) 

5,177 news on holidays

In [ ]:
dict_days={0:"Mon",
          1:"Tue",
          2:"Wed",
          3:"Thu",
          4:"Fri",
          5:"Sat",
          6:"Sun"}


# Insert controll variable for the day of the week (0=Monday, 6=Sunday)
frame["Weekday"]=pd.to_datetime(frame.index).dayofweek.map(dict_days)
frame

In [ ]:
frame[frame.Weekday.isin(["Sat","Sun"])]

34,123 news are on weekends.

**Summary:**

Holliday and Weekend news account for

34,123 + 5,177 = 39,300 news

of 702,039 news articles

39300/702039= 0,5597 = 5.6% of all news